### [Exclusive time of functions](https://leetcode.com/problems/exclusive-time-of-functions/solution/)

Given the running logs of n functions that are executed in a nonpreemptive single threaded CPU, find the exclusive time of these functions.

Each function has a unique id, start from 0 to n-1. A function may be called recursively or by another function.

A log is a string has this format : `function_id:start_or_end:timestamp`. For example, "0:start:0" means function 0 starts from the very beginning of time 0. "0:end:0" means function 0 ends to the very end of time 0.

Exclusive time of a function is defined as the time spent within this function, the time spent by calling other functions should not be considered as this function's exclusive time. You should return the exclusive time of each function sorted by their function id.

**Example 1:**
```
Input:
n = 2
logs = 
["0:start:0",
 "1:start:2",
 "1:end:5",
 "0:end:6"]
Output:[3, 4]
```
**Explanation:**
Function 0 starts at time 0, then it executes 2 units of time and reaches the end of time 1. 
Now function 0 calls function 1, function 1 starts at time 2, executes 4 units of time and end at time 5.
Function 0 is running again at time 6, and also end at the time 6, thus executes 1 unit of time. 
So function 0 totally execute 2 + 1 = 3 units of time, and function 1 totally execute 4 units of time.

**Note:**
- Input logs will be sorted by timestamp, NOT log id.
- Your output should be sorted by function id, which means the 0th element of your output corresponds to the exclusive time of function 0.
- Two functions won't start or end at the same time.
- Functions could be called recursively, and will always end.
- 1 <= n <= 100

In [1]:
class Solution(object):
    def exclusiveTime(self, n, logs):
        """
        :type n: int
        :type logs: List[str]
        :rtype: List[int]
        """
        # funtion logs..sorted by timestamp
        # pattern: 
        #   id: start|end : time
        # function can call other function or itself
        #
        # exclusive time:
        #   time spent within the function. 
        #       time spent in calling other function is not counted.
        #       time spent in calling itself will be counted towards exclusive time
        #
        # n - number of functions
        # return list of exclusive time, sorted by function-id
        #
        # logs sorted by timestamp
        #   single threaded, non-preemptive, so only one function can be running at any time.
        #   running function id can be tracked
        #   start of one function
        #       pause of another function
        #   end of one function
        #       resume of another function
        #   at the min: 1 cycle
        #
        # stack to store the last running function id:?
        # at start, push function id and start time to stack
        # if the stack was not empty before pushing, some other function was
        # running before that. we must update the cycles of that function too.
        # prev running func: exclusive time elapsed = current time - start-time - 1
        # at start:
        #    pop from the stack. update the excl time (inclusive of end time)
        # but note the end time.. or update the stack with end time as start time
        
        # logs are not sorted by function-id.
        # assume each function is executed at least one. so we surely have
        # an entry for that in our exclusive times
        
        # edge cases
        #   0 functions
        #   assume logs are formatted correctly.
        if n == 0:
            return []
        
        exclusive_times = [0] * n
        stack = []
        
        for log in logs:
            func_id, op, timestamp = log.split(":")
            func_id, timestamp = int(func_id), int(timestamp)
            if op == "start":
                if stack:
                    # update the end time and elapsed time of current running
                    # function and push the new function on to the stack
                    curr_func_id, curr_func_start_time = stack[-1]
                    exclusive_times[curr_func_id] += (timestamp - curr_func_start_time)
                    stack.append([func_id, timestamp])
                else:
                    # nothing to do if the stack is empty
                    # even if the function was previously running,
                    # we would have updated its time when it ended
                    # last time.
                    stack.append([func_id, timestamp])
            
            if op == "end":
                # pop from the stack
                _, start_time = stack.pop()
                exclusive_times[func_id] += (timestamp - start_time + 1)
                
                # update the start time of current function in stack
                if stack:
                    stack[-1][1] = timestamp + 1 # adding 1 because next function
                                                 # cannot start at the current time
                
        # stack must be empty as all function calls are
        # guaranteed to end.
        return exclusive_times
    
        # Complexities
        # Time: O(n)
        # Space: O(n/2) -> O(n), if all calls are nested calls.

In [6]:
# sample inputs
test_inputs = {
    "test" : {
        "n" : 2,
        "logs" : ["0:start:0","1:start:2","1:start:4","1:end:5","1:end:7", "0:end:9","1:start:11","1:end:12"],
        "output" : [4,8]
    },
    "test" : {
        "n" : 3,
        "logs" : ["0:start:0","2:start:2","1:start:4","1:end:5","2:end:7", "0:end:9"],
        "output" : [4, 2, 4]
    }
}

s = Solution()
for test_input in test_inputs.values():
    n, logs = test_input["n"], test_input["logs"]
    assert(s.exclusiveTime(n, logs) == test_input["output"])
    